In [1]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load in 

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the "../input/" directory.
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# Any results you write to the current directory are saved as output.

/kaggle/input/test.csv
/kaggle/input/train.csv


In [2]:
import math
np.set_printoptions(suppress=True)

import warnings
warnings.filterwarnings('ignore')

GAME_TRAINING = "/kaggle/input/train.csv"
GAME_TEST = "/kaggle/input/test.csv"

train_data = pd.read_csv(GAME_TRAINING)
pd.options.display.max_columns = 100

test_data = pd.read_csv(GAME_TEST)
pd.options.display.max_columns = 100

In [3]:
train_data

,id,playtime_forever,is_free,price,genres,categories,tags,purchase_date,release_date,total_positive_reviews,total_negative_reviews
0,0,0.000000,False,3700.0,"Adventure,Casual,Indie","Single-player,Steam Trading Cards,Steam Cloud","Indie,Adventure,Story Rich,Casual,Atmospheric,...","Jul 2, 2018","10 Dec, 2013",372.0,96.0
1,1,0.016667,True,0.0,RPG,"Single-player,Partial Controller Support","Mod,Utilities,RPG,Game Development,Singleplaye...","Nov 26, 2016","12 Aug, 2015",23.0,0.0
2,2,0.000000,False,5000.0,"Adventure,Casual,Indie","Single-player,Full controller support,Steam Tr...","Point & Click,Adventure,Story Rich,Comedy,Indi...","Jul 2, 2018","28 Jan, 2014",3018.0,663.0
3,3,1.533333,False,9900.0,"Action,RPG","Single-player,Multi-player,Steam Achievements,...","Medieval,RPG,Open World,Strategy,Sandbox,Actio...","Nov 28, 2016","31 Mar, 2010",63078.0,1746.0
4,4,22.333333,False,4800.0,"Action,Indie,Strategy","Single-player,Co-op,Steam Achievements,Full co...","Tower Defense,Co-op,Action,Strategy,Online Co-...","Mar 4, 2018","30 Jul, 2012",8841.0,523.0
...,...,...,...,...,...,...,...,...,...,...,...
352,352,0.783333,False,8800.0,"Action,Simulation","Single-player,Multi-player,Online Multi-Player...","Simulation,Action,VR,Flight","Nov 24, 2017","20 Dec, 2016",150.0,91.0
353,353,0.000000,False,6800.0,"Action,Adventure","Single-player,Steam Achievements,Full controll...","Batman,Action,Open World,Superhero,Stealth,Adv...","Aug 15, 2018","23 Jun, 2015",19008.0,4849.0
354,354,0.000000,False,8300.0,"Action,Adventure,RPG","Single-player,Steam Achievements,Full controll...","Action,Hack and Slash,Adventure,RPG,Open World...","Jan 30, 2018","5 Nov, 2015",5099.0,1719.0
355,355,0.000000,False,6800.0,"Action,Adventure","Single-player,Steam Achievements,Full controll...","Action,FPS,Adventure,Shooter,Cyberpunk,Robots,...","Sep 23, 2017","3 Jun, 2016",718.0,159.0


In [4]:
# Train dataset, drop some columns i dont need
drop_column=['categories','tags']
train_data_new = train_data.drop(columns=drop_column)
train_data_new

,id,playtime_forever,is_free,price,genres,purchase_date,release_date,total_positive_reviews,total_negative_reviews
0,0,0.000000,False,3700.0,"Adventure,Casual,Indie","Jul 2, 2018","10 Dec, 2013",372.0,96.0
1,1,0.016667,True,0.0,RPG,"Nov 26, 2016","12 Aug, 2015",23.0,0.0
2,2,0.000000,False,5000.0,"Adventure,Casual,Indie","Jul 2, 2018","28 Jan, 2014",3018.0,663.0
3,3,1.533333,False,9900.0,"Action,RPG","Nov 28, 2016","31 Mar, 2010",63078.0,1746.0
4,4,22.333333,False,4800.0,"Action,Indie,Strategy","Mar 4, 2018","30 Jul, 2012",8841.0,523.0
...,...,...,...,...,...,...,...,...,...
352,352,0.783333,False,8800.0,"Action,Simulation","Nov 24, 2017","20 Dec, 2016",150.0,91.0
353,353,0.000000,False,6800.0,"Action,Adventure","Aug 15, 2018","23 Jun, 2015",19008.0,4849.0
354,354,0.000000,False,8300.0,"Action,Adventure,RPG","Jan 30, 2018","5 Nov, 2015",5099.0,1719.0
355,355,0.000000,False,6800.0,"Action,Adventure","Sep 23, 2017","3 Jun, 2016",718.0,159.0


In [5]:
# Test dataset, drop some columns i dont need
drop_column=['categories','tags']
test_data_new = test_data.drop(columns=drop_column)
test_data_new

,id,is_free,price,genres,purchase_date,release_date,total_positive_reviews,total_negative_reviews
0,0,False,3500,"Action,Adventure","Oct 28, 2018",22-May-12,2607.0,1122.0
1,1,False,11600,"Action,Adventure,Strategy","Jul 20, 2019",24-Apr-18,5762.0,2235.0
2,2,False,2100,"Indie,Simulation,Strategy","Jul 16, 2019",20-Mar-17,687.0,133.0
3,3,False,3600,"Action,Strategy","Jan 30, 2018",17-Jul-07,67.0,39.0
4,4,False,3400,"Action,Adventure","Feb 24, 2017",22-Aug-13,40344.0,3708.0
...,...,...,...,...,...,...,...,...
85,85,False,5000,Action,"Mar 23, 2018",22-May-13,9334.0,516.0
86,86,False,3600,"Action,Adventure","Mar 3, 2018",14-Sep-09,1245.0,176.0
87,87,False,11200,"Simulation,Strategy","Feb 18, 2018",13-Aug-13,33128.0,6350.0
88,88,True,0,"Action,Adventure,RPG,Simulation,Sports,Strategy","Nov 19, 2017",19-Jun-17,5.0,0.0


In [6]:
# check the null values
train_data_new.isna().sum()

id                        0
playtime_forever          0
is_free                   0
price                     0
genres                    0
purchase_date             2
release_date              0
total_positive_reviews    2
total_negative_reviews    2
dtype: int64

In [7]:
# train dataset, transform the null to mean value
train_data_new.total_positive_reviews[train_data_new.total_positive_reviews.isnull()] \
    = train_data_new.total_positive_reviews.dropna().mean()
train_data_new.total_negative_reviews[train_data_new.total_negative_reviews.isnull()] \
    = train_data_new.total_negative_reviews.dropna().mean()
train_data_new.purchase_date[train_data_new.purchase_date.isnull()] \
    = train_data_new.purchase_date.dropna()
train_data_new.isna().sum()

id                        0
playtime_forever          0
is_free                   0
price                     0
genres                    0
purchase_date             2
release_date              0
total_positive_reviews    0
total_negative_reviews    0
dtype: int64

In [8]:
test_data_new.isna().sum()

id                        0
is_free                   0
price                     0
genres                    0
purchase_date             2
release_date              0
total_positive_reviews    2
total_negative_reviews    2
dtype: int64

In [9]:
# test dataset, transform the null to mean value
test_data_new.total_positive_reviews[test_data_new.total_positive_reviews.isnull()] \
    = test_data_new.total_positive_reviews.dropna().mean()
test_data_new.total_negative_reviews[test_data_new.total_negative_reviews.isnull()] \
    = test_data_new.total_negative_reviews.dropna().mean()
test_data_new.purchase_date[test_data_new.purchase_date.isnull()] \
    = test_data_new.purchase_date.dropna()
test_data_new.isna().sum()

id                        0
is_free                   0
price                     0
genres                    0
purchase_date             2
release_date              0
total_positive_reviews    0
total_negative_reviews    0
dtype: int64

In [10]:
# check and deal with outliers
train_data_new['price'][train_data_new['price'] > 80000].count()

2

In [11]:
# delete the 2 records with price larger than 80000
train_data_new = train_data_new[train_data_new['price'] < 80000]

In [12]:
# is_free: factorize
train_data_new["is_free"] = pd.factorize(train_data_new["is_free"])[0].astype(np.uint16)
train_data_new["is_free"]

0      0
1      1
2      0
3      0
4      0
      ..
352    0
353    0
354    0
355    0
356    0
Name: is_free, Length: 355, dtype: uint16

In [13]:
test_data_new["is_free"] = pd.factorize(test_data_new["is_free"])[0].astype(np.uint16)
test_data_new["is_free"]

0     0
1     0
2     0
3     0
4     0
     ..
85    0
86    0
87    0
88    1
89    0
Name: is_free, Length: 90, dtype: uint16

In [14]:
# purchase_date/release_date: to_datetime
train_data_new['purchase_date'] = pd.to_datetime(train_data_new['purchase_date'])
train_data_new['release_date'] = pd.to_datetime(train_data_new['release_date'])

# add a column "data_gap", which is the value of purchase_date-release_date
train_data_new['date_gap'] = (train_data_new['purchase_date'] - train_data_new['release_date']) \
                                .astype(str).str.split(' ',expand=True)[0]
train_data_new

,id,playtime_forever,is_free,price,genres,purchase_date,release_date,total_positive_reviews,total_negative_reviews,date_gap
0,0,0.000000,0,3700.0,"Adventure,Casual,Indie",2018-07-02,2013-12-10,372.0,96.0,1665
1,1,0.016667,1,0.0,RPG,2016-11-26,2015-08-12,23.0,0.0,472
2,2,0.000000,0,5000.0,"Adventure,Casual,Indie",2018-07-02,2014-01-28,3018.0,663.0,1616
3,3,1.533333,0,9900.0,"Action,RPG",2016-11-28,2010-03-31,63078.0,1746.0,2434
4,4,22.333333,0,4800.0,"Action,Indie,Strategy",2018-03-04,2012-07-30,8841.0,523.0,2043
...,...,...,...,...,...,...,...,...,...,...
352,352,0.783333,0,8800.0,"Action,Simulation",2017-11-24,2016-12-20,150.0,91.0,339
353,353,0.000000,0,6800.0,"Action,Adventure",2018-08-15,2015-06-23,19008.0,4849.0,1149
354,354,0.000000,0,8300.0,"Action,Adventure,RPG",2018-01-30,2015-11-05,5099.0,1719.0,817
355,355,0.000000,0,6800.0,"Action,Adventure",2017-09-23,2016-06-03,718.0,159.0,477


In [15]:
test_data_new['purchase_date'] = pd.to_datetime(test_data_new['purchase_date'])
test_data_new['release_date'] = pd.to_datetime(test_data_new['release_date'])
test_data_new['date_gap'] = (test_data_new['purchase_date'] - test_data_new['release_date']) \
                                .astype(str).str.split(' ',expand=True)[0]
test_data_new

,id,is_free,price,genres,purchase_date,release_date,total_positive_reviews,total_negative_reviews,date_gap
0,0,0,3500,"Action,Adventure",2018-10-28,2012-05-22,2607.0,1122.0,2350
1,1,0,11600,"Action,Adventure,Strategy",2019-07-20,2018-04-24,5762.0,2235.0,452
2,2,0,2100,"Indie,Simulation,Strategy",2019-07-16,2017-03-20,687.0,133.0,848
3,3,0,3600,"Action,Strategy",2018-01-30,2007-07-17,67.0,39.0,3850
4,4,0,3400,"Action,Adventure",2017-02-24,2013-08-22,40344.0,3708.0,1282
...,...,...,...,...,...,...,...,...,...
85,85,0,5000,Action,2018-03-23,2013-05-22,9334.0,516.0,1766
86,86,0,3600,"Action,Adventure",2018-03-03,2009-09-14,1245.0,176.0,3092
87,87,0,11200,"Simulation,Strategy",2018-02-18,2013-08-13,33128.0,6350.0,1650
88,88,1,0,"Action,Adventure,RPG,Simulation,Sports,Strategy",2017-11-19,2017-06-19,5.0,0.0,153


In [16]:
# deal with some special situations
train_data_new.date_gap[train_data_new['date_gap'] == 'NaT'] = 0
test_data_new.date_gap[test_data_new['date_gap'] == 'NaT'] = 0

day_gap_mean = (train_data_new['date_gap'].astype('float')+train_data_new['date_gap'].astype('float')).mean()
train_data_new.date_gap[train_data_new['date_gap'] == 0] = day_gap_mean
test_data_new.date_gap[test_data_new['date_gap'] == 0] = day_gap_mean

In [17]:
# transform purchase_date to year,month,day columns respectively
train_data_new['purchase_year'] = train_data_new['purchase_date'].astype(str).str.split('-',expand=True)[0]
train_data_new['purchase_month'] = train_data_new['purchase_date'].astype(str).str.split('-',expand=True)[1]
train_data_new['purchase_day'] = train_data_new['purchase_date'].astype(str).str.split('-',expand=True)[2]

test_data_new['purchase_year'] = test_data_new['purchase_date'].astype(str).str.split('-',expand=True)[0]
test_data_new['purchase_month'] = test_data_new['purchase_date'].astype(str).str.split('-',expand=True)[1]
test_data_new['purchase_day'] = test_data_new['purchase_date'].astype(str).str.split('-',expand=True)[2]

train_data_new.drop(["purchase_date"], axis=1, inplace=True)
test_data_new.drop(["purchase_date"], axis=1, inplace=True)
train_data_new

,id,playtime_forever,is_free,price,genres,release_date,total_positive_reviews,total_negative_reviews,date_gap,purchase_year,purchase_month,purchase_day
0,0,0.000000,0,3700.0,"Adventure,Casual,Indie",2013-12-10,372.0,96.0,1665,2018,07,02
1,1,0.016667,1,0.0,RPG,2015-08-12,23.0,0.0,472,2016,11,26
2,2,0.000000,0,5000.0,"Adventure,Casual,Indie",2014-01-28,3018.0,663.0,1616,2018,07,02
3,3,1.533333,0,9900.0,"Action,RPG",2010-03-31,63078.0,1746.0,2434,2016,11,28
4,4,22.333333,0,4800.0,"Action,Indie,Strategy",2012-07-30,8841.0,523.0,2043,2018,03,04
...,...,...,...,...,...,...,...,...,...,...,...,...
352,352,0.783333,0,8800.0,"Action,Simulation",2016-12-20,150.0,91.0,339,2017,11,24
353,353,0.000000,0,6800.0,"Action,Adventure",2015-06-23,19008.0,4849.0,1149,2018,08,15
354,354,0.000000,0,8300.0,"Action,Adventure,RPG",2015-11-05,5099.0,1719.0,817,2018,01,30
355,355,0.000000,0,6800.0,"Action,Adventure",2016-06-03,718.0,159.0,477,2017,09,23


In [18]:
train_data_new.isna().sum()

id                        0
playtime_forever          0
is_free                   0
price                     0
genres                    0
release_date              0
total_positive_reviews    0
total_negative_reviews    0
date_gap                  0
purchase_year             0
purchase_month            2
purchase_day              2
dtype: int64

In [19]:
# deal with the null values of 3 new date columns: be the mean of not-NaT values
train_data_withoutNaT = train_data_new[train_data_new['purchase_year']!='NaT']
train_data_new.purchase_year[train_data_new['purchase_year']=='NaT'] \
    = train_data_withoutNaT.purchase_year.astype(float).mean()
train_data_new.purchase_month[train_data_new.purchase_month.isnull()] \
    = train_data_withoutNaT.purchase_month.astype(float).mean()
train_data_new.purchase_day[train_data_new.purchase_day.isnull()] \
    = train_data_withoutNaT.purchase_day.astype(float).mean()

test_data_withoutNaT = test_data_new[test_data_new['purchase_year']!='NaT']
test_data_new.purchase_year[test_data_new['purchase_year']=='NaT'] \
    = test_data_withoutNaT.purchase_year.astype(float).mean()
test_data_new.purchase_month[test_data_new.purchase_month.isnull()] \
    = test_data_withoutNaT.purchase_month.astype(float).mean()
test_data_new.purchase_day[test_data_new.purchase_day.isnull()] \
    = test_data_withoutNaT.purchase_day.astype(float).mean()
test_data_new.isnull().sum()

id                        0
is_free                   0
price                     0
genres                    0
release_date              0
total_positive_reviews    0
total_negative_reviews    0
date_gap                  0
purchase_year             0
purchase_month            0
purchase_day              0
dtype: int64

In [20]:
train_data_new['date_gap'] = train_data_new['date_gap'].astype(float)
train_data_new['purchase_year'] = train_data_new['purchase_year'].astype(float)
train_data_new['purchase_month'] = train_data_new['purchase_month'].astype(float)
train_data_new['purchase_day'] = train_data_new['purchase_day'].astype(float)

test_data_new['date_gap'] = test_data_new['date_gap'].astype(float)
test_data_new['purchase_year'] = test_data_new['purchase_year'].astype(float)
test_data_new['purchase_month'] = test_data_new['purchase_month'].astype(float)
test_data_new['purchase_day'] = test_data_new['purchase_day'].astype(float)

In [21]:
# release_date
train_data_new['release_year'] = train_data_new['release_date'].astype(str).str.split('-',expand=True)[0]
train_data_new['release_month'] = train_data_new['release_date'].astype(str).str.split('-',expand=True)[1]
train_data_new['release_day'] = train_data_new['release_date'].astype(str).str.split('-',expand=True)[2]

test_data_new['release_year'] = test_data_new['release_date'].astype(str).str.split('-',expand=True)[0]
test_data_new['release_month'] = test_data_new['release_date'].astype(str).str.split('-',expand=True)[1]
test_data_new['release_day'] = test_data_new['release_date'].astype(str).str.split('-',expand=True)[2]

train_data_new.drop(["release_date"], axis=1, inplace=True)
test_data_new.drop(["release_date"], axis=1, inplace=True)

In [22]:
train_data_withoutNaT = train_data_new[train_data_new['release_year']!='NaT']
train_data_new.release_year[train_data_new['release_year']=='NaT'] \
    = train_data_withoutNaT.release_year.astype(float).mean()
train_data_new.release_month[train_data_new.release_month.isnull()] \
    = train_data_withoutNaT.release_month.astype(float).mean()
train_data_new.release_day[train_data_new.release_day.isnull()] \
    = train_data_withoutNaT.release_day.astype(float).mean()

test_data_withoutNaT = test_data_new[test_data_new['release_year']!='NaT']
test_data_new.release_year[test_data_new['release_year']=='NaT'] \
    = test_data_withoutNaT.release_year.astype(float).mean()
test_data_new.release_month[test_data_new.release_month.isnull()] \
    = test_data_withoutNaT.release_month.astype(float).mean()
test_data_new.release_day[test_data_new.release_day.isnull()] \
    = test_data_withoutNaT.release_day.astype(float).mean()

In [23]:
train_data_new['release_year'] = train_data_new['release_year'].astype(float)
train_data_new['release_month'] = train_data_new['release_month'].astype(float)
train_data_new['release_day'] = train_data_new['release_day'].astype(float)

test_data_new['release_year'] = test_data_new['release_year'].astype(float)
test_data_new['release_month'] = test_data_new['release_month'].astype(float)
test_data_new['release_day'] = test_data_new['release_day'].astype(float)

In [24]:
# add a column "posratio", which is total_positive_reviews/(total_positive_reviews+total_negative_reviews)
train_data_new['posratio'] = (train_data_new['total_positive_reviews'] / \
                            (train_data_new['total_positive_reviews'] + train_data_new['total_negative_reviews']))
train_data_new

,id,playtime_forever,is_free,price,genres,total_positive_reviews,total_negative_reviews,date_gap,purchase_year,purchase_month,purchase_day,release_year,release_month,release_day,posratio
0,0,0.000000,0,3700.0,"Adventure,Casual,Indie",372.0,96.0,1665.0,2018.0,7.0,2.0,2013.0,12.0,10.0,0.794872
1,1,0.016667,1,0.0,RPG,23.0,0.0,472.0,2016.0,11.0,26.0,2015.0,8.0,12.0,1.000000
2,2,0.000000,0,5000.0,"Adventure,Casual,Indie",3018.0,663.0,1616.0,2018.0,7.0,2.0,2014.0,1.0,28.0,0.819886
3,3,1.533333,0,9900.0,"Action,RPG",63078.0,1746.0,2434.0,2016.0,11.0,28.0,2010.0,3.0,31.0,0.973066
4,4,22.333333,0,4800.0,"Action,Indie,Strategy",8841.0,523.0,2043.0,2018.0,3.0,4.0,2012.0,7.0,30.0,0.944148
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
352,352,0.783333,0,8800.0,"Action,Simulation",150.0,91.0,339.0,2017.0,11.0,24.0,2016.0,12.0,20.0,0.622407
353,353,0.000000,0,6800.0,"Action,Adventure",19008.0,4849.0,1149.0,2018.0,8.0,15.0,2015.0,6.0,23.0,0.796747
354,354,0.000000,0,8300.0,"Action,Adventure,RPG",5099.0,1719.0,817.0,2018.0,1.0,30.0,2015.0,11.0,5.0,0.747873
355,355,0.000000,0,6800.0,"Action,Adventure",718.0,159.0,477.0,2017.0,9.0,23.0,2016.0,6.0,3.0,0.818700


In [25]:
train_data_new['posratio'].sort_values()

121    0.227224
12     0.297297
19     0.391534
136    0.397263
286    0.417021
         ...   
65          NaN
107         NaN
127         NaN
200         NaN
322         NaN
Name: posratio, Length: 355, dtype: float64

In [26]:
train_data_new.posratio[train_data_new.posratio.isnull()] \
    = train_data_new.posratio.dropna().mean()
train_data_new['posratio']

0      0.794872
1      1.000000
2      0.819886
3      0.973066
4      0.944148
         ...   
352    0.622407
353    0.796747
354    0.747873
355    0.818700
356    0.899705
Name: posratio, Length: 355, dtype: float64

In [27]:
test_data_new['posratio'] = (test_data_new['total_positive_reviews'] / \
                   (test_data_new['total_positive_reviews'] + test_data_new['total_negative_reviews']))
test_data_new

,id,is_free,price,genres,total_positive_reviews,total_negative_reviews,date_gap,purchase_year,purchase_month,purchase_day,release_year,release_month,release_day,posratio
0,0,0,3500,"Action,Adventure",2607.0,1122.0,2350.0,2018.0,10.0,28.0,2012.0,5.0,22.0,0.699115
1,1,0,11600,"Action,Adventure,Strategy",5762.0,2235.0,452.0,2019.0,7.0,20.0,2018.0,4.0,24.0,0.720520
2,2,0,2100,"Indie,Simulation,Strategy",687.0,133.0,848.0,2019.0,7.0,16.0,2017.0,3.0,20.0,0.837805
3,3,0,3600,"Action,Strategy",67.0,39.0,3850.0,2018.0,1.0,30.0,2007.0,7.0,17.0,0.632075
4,4,0,3400,"Action,Adventure",40344.0,3708.0,1282.0,2017.0,2.0,24.0,2013.0,8.0,22.0,0.915827
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
85,85,0,5000,Action,9334.0,516.0,1766.0,2018.0,3.0,23.0,2013.0,5.0,22.0,0.947614
86,86,0,3600,"Action,Adventure",1245.0,176.0,3092.0,2018.0,3.0,3.0,2009.0,9.0,14.0,0.876144
87,87,0,11200,"Simulation,Strategy",33128.0,6350.0,1650.0,2018.0,2.0,18.0,2013.0,8.0,13.0,0.839151
88,88,1,0,"Action,Adventure,RPG,Simulation,Sports,Strategy",5.0,0.0,153.0,2017.0,11.0,19.0,2017.0,6.0,19.0,1.000000


In [28]:
test_data_new.posratio[test_data_new.posratio.isnull()] \
    = test_data_new.posratio.dropna().mean()
test_data_new['posratio']

0     0.699115
1     0.720520
2     0.837805
3     0.632075
4     0.915827
        ...   
85    0.947614
86    0.876144
87    0.839151
88    1.000000
89    0.872662
Name: posratio, Length: 90, dtype: float64

In [29]:
# generes: dummies to sparse matrix
d_genres = train_data_new["genres"].str.get_dummies(",")

# bring the dummies back into the original dataset
train_data_new.drop(["genres"], axis=1, inplace=True)
train_data_new = pd.concat([train_data_new, d_genres], axis=1)
train_data_new

,id,playtime_forever,is_free,price,total_positive_reviews,total_negative_reviews,date_gap,purchase_year,purchase_month,purchase_day,release_year,release_month,release_day,posratio,Action,Adventure,Animation & Modeling,Audio Production,Casual,Design & Illustration,Early Access,Free to Play,Gore,Indie,Massively Multiplayer,Nudity,RPG,Racing,Sexual Content,Simulation,Sports,Strategy,Utilities,Violent
0,0,0.000000,0,3700.0,372.0,96.0,1665.0,2018.0,7.0,2.0,2013.0,12.0,10.0,0.794872,0,1,0,0,1,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0
1,1,0.016667,1,0.0,23.0,0.0,472.0,2016.0,11.0,26.0,2015.0,8.0,12.0,1.000000,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0
2,2,0.000000,0,5000.0,3018.0,663.0,1616.0,2018.0,7.0,2.0,2014.0,1.0,28.0,0.819886,0,1,0,0,1,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0
3,3,1.533333,0,9900.0,63078.0,1746.0,2434.0,2016.0,11.0,28.0,2010.0,3.0,31.0,0.973066,1,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0
4,4,22.333333,0,4800.0,8841.0,523.0,2043.0,2018.0,3.0,4.0,2012.0,7.0,30.0,0.944148,1,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,1,0,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
352,352,0.783333,0,8800.0,150.0,91.0,339.0,2017.0,11.0,24.0,2016.0,12.0,20.0,0.622407,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0
353,353,0.000000,0,6800.0,19008.0,4849.0,1149.0,2018.0,8.0,15.0,2015.0,6.0,23.0,0.796747,1,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0
354,354,0.000000,0,8300.0,5099.0,1719.0,817.0,2018.0,1.0,30.0,2015.0,11.0,5.0,0.747873,1,1,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0
355,355,0.000000,0,6800.0,718.0,159.0,477.0,2017.0,9.0,23.0,2016.0,6.0,3.0,0.818700,1,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0


In [30]:
d_genres = test_data_new["genres"].str.get_dummies(",")

# bring the dummies back into the original dataset
test_data_new.drop(["genres"], axis=1, inplace=True)
test_data_new = pd.concat([test_data_new, d_genres], axis=1)
test_data_new

,id,is_free,price,total_positive_reviews,total_negative_reviews,date_gap,purchase_year,purchase_month,purchase_day,release_year,release_month,release_day,posratio,Action,Adventure,Casual,Early Access,Free to Play,Gore,Indie,Massively Multiplayer,Nudity,RPG,Simulation,Sports,Strategy,Violent
0,0,0,3500,2607.0,1122.0,2350.0,2018.0,10.0,28.0,2012.0,5.0,22.0,0.699115,1,1,0,0,0,0,0,0,0,0,0,0,0,0
1,1,0,11600,5762.0,2235.0,452.0,2019.0,7.0,20.0,2018.0,4.0,24.0,0.720520,1,1,0,0,0,0,0,0,0,0,0,0,1,0
2,2,0,2100,687.0,133.0,848.0,2019.0,7.0,16.0,2017.0,3.0,20.0,0.837805,0,0,0,0,0,0,1,0,0,0,1,0,1,0
3,3,0,3600,67.0,39.0,3850.0,2018.0,1.0,30.0,2007.0,7.0,17.0,0.632075,1,0,0,0,0,0,0,0,0,0,0,0,1,0
4,4,0,3400,40344.0,3708.0,1282.0,2017.0,2.0,24.0,2013.0,8.0,22.0,0.915827,1,1,0,0,0,0,0,0,0,0,0,0,0,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
85,85,0,5000,9334.0,516.0,1766.0,2018.0,3.0,23.0,2013.0,5.0,22.0,0.947614,1,0,0,0,0,0,0,0,0,0,0,0,0,0
86,86,0,3600,1245.0,176.0,3092.0,2018.0,3.0,3.0,2009.0,9.0,14.0,0.876144,1,1,0,0,0,0,0,0,0,0,0,0,0,0
87,87,0,11200,33128.0,6350.0,1650.0,2018.0,2.0,18.0,2013.0,8.0,13.0,0.839151,0,0,0,0,0,0,0,0,0,0,1,0,1,0
88,88,1,0,5.0,0.0,153.0,2017.0,11.0,19.0,2017.0,6.0,19.0,1.000000,1,1,0,0,0,0,0,0,0,1,1,1,1,0


In [31]:
train_data_new

,id,playtime_forever,is_free,price,total_positive_reviews,total_negative_reviews,date_gap,purchase_year,purchase_month,purchase_day,release_year,release_month,release_day,posratio,Action,Adventure,Animation & Modeling,Audio Production,Casual,Design & Illustration,Early Access,Free to Play,Gore,Indie,Massively Multiplayer,Nudity,RPG,Racing,Sexual Content,Simulation,Sports,Strategy,Utilities,Violent
0,0,0.000000,0,3700.0,372.0,96.0,1665.0,2018.0,7.0,2.0,2013.0,12.0,10.0,0.794872,0,1,0,0,1,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0
1,1,0.016667,1,0.0,23.0,0.0,472.0,2016.0,11.0,26.0,2015.0,8.0,12.0,1.000000,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0
2,2,0.000000,0,5000.0,3018.0,663.0,1616.0,2018.0,7.0,2.0,2014.0,1.0,28.0,0.819886,0,1,0,0,1,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0
3,3,1.533333,0,9900.0,63078.0,1746.0,2434.0,2016.0,11.0,28.0,2010.0,3.0,31.0,0.973066,1,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0
4,4,22.333333,0,4800.0,8841.0,523.0,2043.0,2018.0,3.0,4.0,2012.0,7.0,30.0,0.944148,1,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,1,0,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
352,352,0.783333,0,8800.0,150.0,91.0,339.0,2017.0,11.0,24.0,2016.0,12.0,20.0,0.622407,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0
353,353,0.000000,0,6800.0,19008.0,4849.0,1149.0,2018.0,8.0,15.0,2015.0,6.0,23.0,0.796747,1,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0
354,354,0.000000,0,8300.0,5099.0,1719.0,817.0,2018.0,1.0,30.0,2015.0,11.0,5.0,0.747873,1,1,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0
355,355,0.000000,0,6800.0,718.0,159.0,477.0,2017.0,9.0,23.0,2016.0,6.0,3.0,0.818700,1,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0


In [32]:
test_data_new

,id,is_free,price,total_positive_reviews,total_negative_reviews,date_gap,purchase_year,purchase_month,purchase_day,release_year,release_month,release_day,posratio,Action,Adventure,Casual,Early Access,Free to Play,Gore,Indie,Massively Multiplayer,Nudity,RPG,Simulation,Sports,Strategy,Violent
0,0,0,3500,2607.0,1122.0,2350.0,2018.0,10.0,28.0,2012.0,5.0,22.0,0.699115,1,1,0,0,0,0,0,0,0,0,0,0,0,0
1,1,0,11600,5762.0,2235.0,452.0,2019.0,7.0,20.0,2018.0,4.0,24.0,0.720520,1,1,0,0,0,0,0,0,0,0,0,0,1,0
2,2,0,2100,687.0,133.0,848.0,2019.0,7.0,16.0,2017.0,3.0,20.0,0.837805,0,0,0,0,0,0,1,0,0,0,1,0,1,0
3,3,0,3600,67.0,39.0,3850.0,2018.0,1.0,30.0,2007.0,7.0,17.0,0.632075,1,0,0,0,0,0,0,0,0,0,0,0,1,0
4,4,0,3400,40344.0,3708.0,1282.0,2017.0,2.0,24.0,2013.0,8.0,22.0,0.915827,1,1,0,0,0,0,0,0,0,0,0,0,0,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
85,85,0,5000,9334.0,516.0,1766.0,2018.0,3.0,23.0,2013.0,5.0,22.0,0.947614,1,0,0,0,0,0,0,0,0,0,0,0,0,0
86,86,0,3600,1245.0,176.0,3092.0,2018.0,3.0,3.0,2009.0,9.0,14.0,0.876144,1,1,0,0,0,0,0,0,0,0,0,0,0,0
87,87,0,11200,33128.0,6350.0,1650.0,2018.0,2.0,18.0,2013.0,8.0,13.0,0.839151,0,0,0,0,0,0,0,0,0,0,1,0,1,0
88,88,1,0,5.0,0.0,153.0,2017.0,11.0,19.0,2017.0,6.0,19.0,1.000000,1,1,0,0,0,0,0,0,0,1,1,1,1,0


In [33]:
y_train = train_data_new['playtime_forever']
X_train = train_data_new.drop(columns=['playtime_forever'])
X_test = test_data_new
print("X_train:",X_train.shape)
print("y_train:",y_train.shape)
print("X_test:",X_test.shape)

X_train: (355, 33)
y_train: (355,)
X_test: (90, 27)


In [34]:
# add the 6 missing columns led from dummies of generes in test dataset
X_test.insert(15, 'Animation & Modeling', 0)
X_test.insert(16, 'Audio Production', 0)
X_test.insert(18, 'Design & Illustration', 0)
X_test.insert(26, 'Racing', 0)
X_test.insert(27, 'Sexual Content', 0)
X_test.insert(31, 'Utilities', 0)
X_test

,id,is_free,price,total_positive_reviews,total_negative_reviews,date_gap,purchase_year,purchase_month,purchase_day,release_year,release_month,release_day,posratio,Action,Adventure,Animation & Modeling,Audio Production,Casual,Design & Illustration,Early Access,Free to Play,Gore,Indie,Massively Multiplayer,Nudity,RPG,Racing,Sexual Content,Simulation,Sports,Strategy,Utilities,Violent
0,0,0,3500,2607.0,1122.0,2350.0,2018.0,10.0,28.0,2012.0,5.0,22.0,0.699115,1,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0
1,1,0,11600,5762.0,2235.0,452.0,2019.0,7.0,20.0,2018.0,4.0,24.0,0.720520,1,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0
2,2,0,2100,687.0,133.0,848.0,2019.0,7.0,16.0,2017.0,3.0,20.0,0.837805,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,1,0,1,0,0
3,3,0,3600,67.0,39.0,3850.0,2018.0,1.0,30.0,2007.0,7.0,17.0,0.632075,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0
4,4,0,3400,40344.0,3708.0,1282.0,2017.0,2.0,24.0,2013.0,8.0,22.0,0.915827,1,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
85,85,0,5000,9334.0,516.0,1766.0,2018.0,3.0,23.0,2013.0,5.0,22.0,0.947614,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0
86,86,0,3600,1245.0,176.0,3092.0,2018.0,3.0,3.0,2009.0,9.0,14.0,0.876144,1,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0
87,87,0,11200,33128.0,6350.0,1650.0,2018.0,2.0,18.0,2013.0,8.0,13.0,0.839151,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,1,0,0
88,88,1,0,5.0,0.0,153.0,2017.0,11.0,19.0,2017.0,6.0,19.0,1.000000,1,1,0,0,0,0,0,0,0,0,0,0,1,0,0,1,1,1,0,0


In [35]:
# normalization(for XGboost)
X_train_norm = (X_train - X_train.min()) / (X_train.max() - X_train.min())
X_test_norm = (X_test - X_test.min()) / (X_test.max() - X_test.min())
X_train_norm

,id,is_free,price,total_positive_reviews,total_negative_reviews,date_gap,purchase_year,purchase_month,purchase_day,release_year,release_month,release_day,posratio,Action,Adventure,Animation & Modeling,Audio Production,Casual,Design & Illustration,Early Access,Free to Play,Gore,Indie,Massively Multiplayer,Nudity,RPG,Racing,Sexual Content,Simulation,Sports,Strategy,Utilities,Violent
0,0.000000,0.0,0.120130,0.000844,0.000220,0.446990,0.75,0.545455,0.033333,0.538462,1.000000,0.300000,0.734557,0.0,1.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
1,0.002809,1.0,0.000000,0.000052,0.000000,0.198500,0.25,0.909091,0.833333,0.692308,0.636364,0.366667,1.000000,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
2,0.005618,0.0,0.162338,0.006845,0.001520,0.436784,0.75,0.545455,0.033333,0.615385,0.000000,0.900000,0.766926,0.0,1.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
3,0.008427,0.0,0.321429,0.143066,0.004004,0.607165,0.25,0.909091,0.900000,0.307692,0.181818,1.000000,0.965146,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
4,0.011236,0.0,0.155844,0.020052,0.001199,0.525724,0.75,0.181818,0.100000,0.461538,0.545455,0.966667,0.927725,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
352,0.988764,0.0,0.285714,0.000340,0.000209,0.170798,0.50,0.909091,0.766667,0.769231,1.000000,0.633333,0.511381,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0
353,0.991573,0.0,0.220779,0.043112,0.011120,0.339513,0.75,0.636364,0.466667,0.692308,0.454545,0.733333,0.736984,1.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
354,0.994382,0.0,0.269481,0.011565,0.003942,0.270360,0.75,0.000000,0.966667,0.692308,0.909091,0.133333,0.673739,1.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
355,0.997191,0.0,0.220779,0.001628,0.000365,0.199542,0.50,0.727273,0.733333,0.769231,0.454545,0.066667,0.765391,1.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0


In [36]:
# XGboost
import xgboost as xgb

xgb_model = xgb.XGBRegressor(booster='gbtree',
                    objective= 'reg:linear',
                    eval_metric='rmse',
                    gamma = 0.1,
                    min_child_weight= 1.1,
                    max_depth= 5,
                    subsample= 0.8,
                    colsample_bytree= 0.8,
                    tree_method= 'exact',
                    learning_rate=0.1,
                    n_estimators=100,
                    nthread=4,
                    scale_pos_weight=1,
                    seed=27)

xgb_model.fit(X_train_norm, y_train)

print('Correlation for xgboost: ', math.sqrt(xgb_model.score(X_train_norm,y_train)))

[10:28:59] WARNING: /workspace/src/objective/regression_obj.cu:152: reg:linear is now deprecated in favor of reg:squarederror.
Correlation for xgboost:  0.9968552373831342


In [37]:
# Predicting result using XGboost
y_pred = xgb_model.predict(X_test)
y_pred

array([10.953894  , 10.095477  , 10.93205   , 10.137201  ,  9.6879835 ,
       13.1283245 , 10.843022  , 10.070469  , 10.003262  ,  8.627396  ,
        9.839202  ,  8.095247  , 12.3367405 , 14.618267  ,  8.179657  ,
        1.5565677 , 11.58652   , 10.610389  , 14.001228  , 12.241223  ,
       10.532206  ,  8.571258  ,  8.928802  , 12.535455  , 10.766225  ,
       14.347833  , 13.114154  , 14.669786  , 13.223841  , 11.662523  ,
       10.521535  , 10.19114   , 10.579191  , 13.223841  ,  9.915044  ,
       13.223841  , 12.384866  , 15.903818  , 14.347833  ,  8.179657  ,
        9.799542  , 19.26659   , 10.521099  , 12.446427  , 13.047345  ,
        9.84382   , 11.4731455 , 13.223841  ,  8.095247  , 10.103231  ,
        0.88287234, 19.26659   ,  9.523446  ,  9.225848  , 12.287297  ,
       10.994241  , 13.047345  , 11.718661  ,  7.9394903 , 13.926443  ,
       13.184463  , 11.258904  , 13.223841  ,  9.743685  , 10.905213  ,
       13.114154  ,  9.065841  , 15.125246  , 10.677197  , 10.69

In [38]:
# RF
from sklearn.ensemble import RandomForestRegressor
regressor = RandomForestRegressor(n_estimators = 10, random_state = 0)
regressor.fit(X_train, y_train)

print('Correlation for RF: ', math.sqrt(regressor.score(X_train,y_train)))

Correlation for RF:  0.8959819675152475


In [39]:
# Predicting result using RF
y_pred = regressor.predict(X_test)
y_pred

array([ 3.10666667,  4.295     ,  2.29166667,  4.46666667,  3.55      ,
        5.47166667,  0.48666667,  0.01666667,  0.49833333,  4.93      ,
        0.95833333,  0.09      ,  0.555     ,  2.395     ,  3.56166667,
        0.62166667,  0.75833333,  1.13333333,  0.38166667,  0.05166667,
        0.835     ,  1.71333333,  0.29666667,  0.475     ,  0.44166667,
        0.57333333,  0.25166667,  0.40666667,  1.09166667,  0.24833333,
        2.14333333, 31.99      ,  2.11      ,  0.45      ,  2.47333333,
        0.14333333,  1.95333333,  3.95833333,  0.11666667,  0.165     ,
        6.25666667,  1.33333333,  1.16      ,  0.44      ,  1.64833333,
        1.86666667,  0.66666667,  0.18      ,  0.46166667, 18.13666667,
        3.22166667,  0.09166667,  1.61333333,  3.695     ,  0.095     ,
        0.87833333, 11.525     ,  0.01833333,  0.26      ,  0.46666667,
        0.81166667,  0.21166667,  0.09166667, 20.66833333,  0.485     ,
        0.82833333,  1.10333333,  0.35833333,  0.575     ,  0.81